In [18]:
'''
1) Прочитайте главы 2-3 из книги "Spark: The Definitive Guide"
2) Установите библиотеки numpy, pandas, matplotlib и seasborn.
Для этого введите в консоли:


source ~/Work/bin/activate
pip3 install numpy
pip3 install pandas
pip3 install matplotlib
pip3 install seaborn

3) Скачайте датасет TikTok_songs_2022.csv по ссылке https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
4) Выполните задания 1-5 с помощью SQL и методов DataFrame.
5) Ответьте на вопросы в задании 6.
'''

'\n1) Прочитайте главы 2-3 из книги "Spark: The Definitive Guide"\n2) Установите библиотеки numpy, pandas, matplotlib и seasborn.\nДля этого введите в консоли:\n\nsource ~/Work/bin/activate\npip3 install numpy\npip3 install pandas\npip3 install matplotlib\npip3 install seaborn\n\n3) Скачайте датасет TikTok_songs_2022.csv по ссылке https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022\n4) Выполните задания 1-5 с помощью SQL и методов DataFrame.\n5) Ответьте на вопросы в задании 6.\n'

In [19]:
# Создаем SparkSession
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkApp') \
                    .getOrCreate()

In [20]:
# Загружаем датасет https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
tiktokData2022 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2022.csv")

tiktokData2022.createOrReplaceTempView("TikTok_songs_2022")

In [21]:
# 0: Получите таблицу из  5 названий треков с их исполнителями.

# +--------------------+-------------+
# |          track_name|  artist_name|
# +--------------------+-------------+
# |Running Up That H...|    Kate Bush|
# |           As It Was| Harry Styles|
# |             Sunroof|  Nicky Youre|
# |          Heat Waves|Glass Animals|
# |     About Damn Time|        Lizzo|
# +--------------------+-------------+

from pyspark.sql.functions import desc

sqlWay = spark.sql("""
SELECT track_name,artist_name
FROM TikTok_songs_2022
LIMIT 5
""")

dataFrameWay = tiktokData2022\
    .select(
    "track_name",
    "artist_name")\
    .limit(5)

sqlWay.show()
dataFrameWay.show()

+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|Running Up That H...|    Kate Bush|
|           As It Was| Harry Styles|
|             Sunroof|  Nicky Youre|
|          Heat Waves|Glass Animals|
|     About Damn Time|        Lizzo|
+--------------------+-------------+

+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|Running Up That H...|    Kate Bush|
|           As It Was| Harry Styles|
|             Sunroof|  Nicky Youre|
|          Heat Waves|Glass Animals|
|     About Damn Time|        Lizzo|
+--------------------+-------------+



In [25]:
# 1: Получите список исполнителей с популярностью больше 90%

# ['Drake', 'Harry Styles', 'The Weeknd']

sqlWay = spark.sql("""
SELECT track_name,artist_name, artist_pop
FROM TikTok_songs_2022 WHERE artist_pop > 90

""")

dataFrameWay =tiktokData2022\
    .select(
    "track_name",
    "artist_name",
    "artist_pop")\
    .where("artist_pop>90")\
    

print(sqlWay.rdd.map(lambda x: x[0]).collect())
print(dataFrameWay.rdd.map(lambda x: x[0]).collect())

sqlWay.show()
dataFrameWay.show()

['As It Was', 'Late Night Talking', 'Blinding Lights', 'Toosie Slide', 'In Your Eyes', 'Blinding Lights', 'Jimmy Cooks (feat. 21 Savage)']
['As It Was', 'Late Night Talking', 'Blinding Lights', 'Toosie Slide', 'In Your Eyes', 'Blinding Lights', 'Jimmy Cooks (feat. 21 Savage)']
+--------------------+------------+----------+
|          track_name| artist_name|artist_pop|
+--------------------+------------+----------+
|           As It Was|Harry Styles|        91|
|  Late Night Talking|Harry Styles|        91|
|     Blinding Lights|  The Weeknd|        93|
|        Toosie Slide|       Drake|        95|
|        In Your Eyes|  The Weeknd|        93|
|     Blinding Lights|  The Weeknd|        93|
|Jimmy Cooks (feat...|       Drake|        95|
+--------------------+------------+----------+

+--------------------+------------+----------+
|          track_name| artist_name|artist_pop|
+--------------------+------------+----------+
|           As It Was|Harry Styles|        91|
|  Late Night Ta

In [26]:
# 2: Выведите список исполнителей и количество их треков в порядке уменьшения.
# Ограничьте выборку исполнителями, у которых не менее 3 треков в наборе данных.

# +--------------+----------------+
# |   artist_name|number_of_tracks|
# +--------------+----------------+
# |      Doja Cat|               8|
# |        Coopex|               6|
# |     Dame Dame|               5|
# |     Lil Nas X|               4|
# |Alex Alexander|               4|
# |       YES YES|               4|
# |         Farux|               4|
# | Ariana Grande|               3|
# |       Astelle|               3|
# |  Alex D'Rosso|               3|
# |      Dua Lipa|               3|
# |    The Weeknd|               3|
# |     Nito-Onna|               3|
# |     Pop Smoke|               3|
# | The Kid LAROI|               3|
# +--------------+----------------+


sqlWay = spark.sql("""
SELECT artist_name, COUNT(artist_name) AS number_of_tracks
FROM TikTok_songs_2022 GROUP BY artist_name HAVING COUNT(artist_name)>2 ORDER BY number_of_tracks DESC
""")

dataFrameWay =tiktokData2022\
    .select("artist_name")\
    .groupBy("artist_name")\
    .count()\
    .withColumnRenamed("count", "number_of_tracks")\
    .sort(desc("number_of_tracks"))\
    .where("number_of_tracks>2")
    

#print(sqlWay.rdd.map(lambda x: x[0]).collect())
#print(dataFrameWay.rdd.map(lambda x: x[0]).collect())

sqlWay.show()
dataFrameWay.show()

+--------------+----------------+
|   artist_name|number_of_tracks|
+--------------+----------------+
|      Doja Cat|               8|
|        Coopex|               6|
|     Dame Dame|               5|
|     Lil Nas X|               4|
|Alex Alexander|               4|
|       YES YES|               4|
|         Farux|               4|
| Ariana Grande|               3|
|       Astelle|               3|
|  Alex D'Rosso|               3|
|      Dua Lipa|               3|
|    The Weeknd|               3|
|     Nito-Onna|               3|
|     Pop Smoke|               3|
| The Kid LAROI|               3|
+--------------+----------------+

+--------------+----------------+
|   artist_name|number_of_tracks|
+--------------+----------------+
|      Doja Cat|               8|
|        Coopex|               6|
|     Dame Dame|               5|
|     Lil Nas X|               4|
|Alex Alexander|               4|
|       YES YES|               4|
|         Farux|               4|
| Ariana Gran

In [29]:
# 3: Выведите список названий треков, средняя громкость которых лежит в диапазоне от -10 до -9 Дб.

# ['Glimpse of Us', 'Left and Right (Charlie Puth) [feat. Jung Kook of BTS] - Sped Up Version',
# 'Frozen', "Can't Get You Out of My Head", 'I Hate U', 'I AM WOMAN', 'Buss It', 'Whatever You Like',
# 'MONEY', 'Freaks', 'telepatía', 'Falling Down', 'Party Girl', 'Skechers', 'Good Looking'] loudness:

sqlWay = spark.sql("""
SELECT track_name
FROM TikTok_songs_2022 WHERE loudness>-10 AND loudness<-9 
""")

dataFrameWay =tiktokData2022\
    .select("track_name")\
    .where("loudness>-10 AND loudness<-9")
    

sqlWay.show()
dataFrameWay.show()

+--------------------+
|          track_name|
+--------------------+
|       Glimpse of Us|
|Left and Right (C...|
|              Frozen|
|Can't Get You Out...|
|            I Hate U|
|          I AM WOMAN|
|             Buss It|
|   Whatever You Like|
|               MONEY|
|              Freaks|
|           telepatía|
|        Falling Down|
|          Party Girl|
|            Skechers|
|        Good Looking|
+--------------------+

+--------------------+
|          track_name|
+--------------------+
|       Glimpse of Us|
|Left and Right (C...|
|              Frozen|
|Can't Get You Out...|
|            I Hate U|
|          I AM WOMAN|
|             Buss It|
|   Whatever You Like|
|               MONEY|
|              Freaks|
|           telepatía|
|        Falling Down|
|          Party Girl|
|            Skechers|
|        Good Looking|
+--------------------+



In [84]:
#4: Выведите список треков, в названиях которых есть слово "love" (без учета регистра).

# ['Another Love', 'love nwantiti (ah ah ah)', 'Tainted Love', 'Looking for Love', 'Free Falling Love',
# 'Electric Love', 'Feel Your Love', 'What You Know Bout Love', 'Savage Love (Laxed - Siren Beat)',
# 'Love Me Again', 'Lovely']

sqlWay = spark.sql("""
SELECT track_name
FROM TikTok_songs_2022 WHERE track_name LIKE '%love%' OR track_name LIKE '%Love%'
""")



dataFrameWay =tiktokData2022\
    .select("track_name")\
    .where("track_name like '%love%' or track_name like '%Love%'")
    

sqlWay.show()
dataFrameWay.show()

+--------------------+
|          track_name|
+--------------------+
|        Another Love|
|love nwantiti (ah...|
|        Tainted Love|
|    Looking for Love|
|   Free Falling Love|
|       Electric Love|
|      Feel Your Love|
|What You Know Bou...|
|Savage Love (Laxe...|
|       Love Me Again|
|              Lovely|
+--------------------+

+--------------------+
|          track_name|
+--------------------+
|        Another Love|
|love nwantiti (ah...|
|        Tainted Love|
|    Looking for Love|
|   Free Falling Love|
|       Electric Love|
|      Feel Your Love|
|What You Know Bou...|
|Savage Love (Laxe...|
|       Love Me Again|
|              Lovely|
+--------------------+



In [91]:
#5: Получите таблицу из песен Арианы Гранде и Astelle. Отсортируйте их по примеру ниже.

# +--------------------+-------------+
# |          track_name|  artist_name|
# +--------------------+-------------+
# |               34+35|Ariana Grande|
# |motive (with Doja...|Ariana Grande|
# |           positions|Ariana Grande|
# |             Dreamer|      Astelle|
# |           K Boy Bye|      Astelle|
# |                Paid|      Astelle|
# +--------------------+-------------+

sqlWay = spark.sql("""
SELECT track_name, artist_name
FROM TikTok_songs_2022 WHERE artist_name LIKE '%Ariana Grande%' OR artist_name LIKE '%Astelle%' ORDER BY artist_name, track_name
""")



dataFrameWay =tiktokData2022\
    .select("track_name", "artist_name")\
    .where("artist_name like '%Astelle%' or artist_name like '%Ariana Grande%'")\
    .sort("artist_name", "track_name")

sqlWay.show()
dataFrameWay.show()

+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|               34+35|Ariana Grande|
|motive (with Doja...|Ariana Grande|
|           positions|Ariana Grande|
|             Dreamer|      Astelle|
|           K Boy Bye|      Astelle|
|                Paid|      Astelle|
+--------------------+-------------+

+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|               34+35|Ariana Grande|
|motive (with Doja...|Ariana Grande|
|           positions|Ariana Grande|
|             Dreamer|      Astelle|
|           K Boy Bye|      Astelle|
|                Paid|      Astelle|
+--------------------+-------------+



In [92]:
#6: Какую задачу решает приведенный ниже код?
# В чем суть метода K средних?
# Какие еще существуют методы машинного обучения для решения данной задачи?
# Какие выводы можно сделать о полученных группах?

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.sql import Row
from pyspark.ml.feature import MinMaxScaler
from IPython.display import display
import numpy as np

k = 20

np.set_printoptions(precision=3, suppress=True)

columns_to_scale = ["danceability", "energy", "speechiness", "acousticness",
                    "instrumentalness","liveness", "valence", "duration_ms",
                    "tempo", "track_pop"]
assemblers = [VectorAssembler(inputCols = [col], outputCol = col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol = col + "_vec", outputCol = col + "_scaled") for col in columns_to_scale]

vectorAssembler = VectorAssembler()\
  .setInputCols(["danceability_scaled", "energy_scaled", "speechiness_scaled", "acousticness_scaled",
                 "instrumentalness_scaled", "liveness_scaled", "valence_scaled", "duration_ms_scaled",
                 "tempo_scaled", "track_pop_scaled"])\
  .setOutputCol("features")

transformationPipeline = Pipeline()\
  .setStages(assemblers + scalers + [vectorAssembler])

fittedPipeline = transformationPipeline.fit(tiktokData2022)
transformedDF = fittedPipeline.transform(tiktokData2022)

kmeans = KMeans()\
  .setK(k)\
  .setSeed(300)

kmModel = kmeans.fit(transformedDF)

df_pred = kmModel.transform(transformedDF)

centers = kmModel.clusterCenters()
for i in range(0, k):
    print("group " + str(i))
    print(centers[i])
    df = df_pred.selectExpr("track_name", "danceability", "energy", "speechiness",
                        "acousticness","instrumentalness","liveness", "valence",
                        "duration_ms", "tempo", "track_pop")\
        .where("prediction = " + str(i))\
        .toPandas()
    display(df)

group 0
[0.745 0.779 0.497 0.064 0.001 0.139 0.551 0.327 0.455 0.582]


,track_name,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,tempo,track_pop
0,Riptide,0.621,0.847,0.278,0.05840,0.000001,0.1090,0.373,172431,101.996,51
1,When You're Gone,0.783,0.753,0.325,0.01170,0.000000,0.1390,0.663,145161,124.036,53
2,THATS WHAT I WANT,0.737,0.846,0.220,0.00614,0.000000,0.0486,0.546,143901,87.981,88
3,We Know,0.871,0.819,0.256,0.14200,0.000000,0.1060,0.712,198982,112.114,39
4,When You Were Young,0.806,0.845,0.190,0.08250,0.000000,0.2600,0.508,128095,125.998,31
5,Bundles (feat. Taylor Girlz),0.939,0.682,0.306,0.00152,0.000000,0.0552,0.584,154806,130.022,67
6,Paid,0.861,0.631,0.230,0.01880,0.000283,0.0954,0.675,180923,117.993,39
7,Up,0.868,0.795,0.269,0.00120,0.019300,0.0461,0.819,156945,166.000,70
8,Best Friend (feat. Doja Cat),0.840,0.766,0.136,0.00302,0.000004,0.0684,0.402,155884,94.018,45
9,Rude Boy,0.739,0.734,0.259,0.02420,0.000000,0.0921,0.471,174858,101.995,50


group 1
[0.59  0.768 0.107 0.077 0.003 0.204 0.433 0.351 0.515 0.547]



[Stage 124:>                                                        (0 + 1) / 1]



,track_name,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,tempo,track_pop
0,Flex On My Ex (with Bankrol Hayden),0.619,0.707,0.0990,0.006010,0.000000,0.0946,0.365,166805,85.285,57
1,That Bxtch,0.645,0.681,0.1390,0.037000,0.000000,0.0783,0.458,142644,163.870,47
2,Warning / Signs,0.712,0.829,0.0300,0.089100,0.027900,0.1680,0.541,155410,121.976,46
3,Serenity,0.688,0.595,0.0508,0.051500,0.000003,0.0853,0.312,202507,117.979,46
4,Tainted Love,0.646,0.765,0.0448,0.043700,0.000141,0.1590,0.362,146667,126.024,47
5,Crazy,0.681,0.865,0.1980,0.282000,0.000169,0.3300,0.494,148293,123.056,53
6,Dinero,0.708,0.782,0.0674,0.007720,0.000000,0.1070,0.484,160880,119.997,73
7,Ocean Drive,0.645,0.868,0.0828,0.066100,0.004710,0.2410,0.510,197000,120.009,51
8,Young Folks,0.750,0.731,0.0379,0.068000,0.000036,0.1840,0.335,174222,134.977,48
9,Complicated,0.724,0.692,0.1090,0.074000,0.000000,0.0937,0.419,152126,134.088,52


...

,track_name,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,tempo,track_pop
0,Left and Right (Charlie Puth) [feat. Jung Kook...,0.827,0.456,0.0942,0.622,0.000098,0.1090,0.689,135978,114.269,58
1,Pope Is a Rockstar,0.725,0.446,0.0287,0.850,0.243000,0.1070,0.311,187333,90.024,74
2,I Hate U,0.535,0.388,0.1550,0.507,0.000000,0.1100,0.412,174000,106.703,79
3,Running Away,0.580,0.696,0.0408,0.660,0.852000,0.1450,0.721,111787,93.688,72
4,Just The Two of Us,0.592,0.214,0.1150,0.936,0.000016,0.2530,0.422,117037,81.672,67
5,Party Girl,0.728,0.431,0.0622,0.749,0.000000,0.0996,0.629,147800,130.022,71
6,Renee,0.924,0.386,0.0548,0.755,0.002530,0.1180,0.424,179125,108.957,65


group 18
[0.564 0.769 0.106 0.187 0.015 0.273 0.146 0.338 0.462 0.534]


,track_name,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,tempo,track_pop
0,Baby Girl,0.730,0.907,0.0606,0.0921,0.114000,0.2300,0.2490,111074,120.968,76
1,Pepas,0.691,0.823,0.0626,0.0235,0.000000,0.0781,0.1490,134330,125.973,58
2,Ocean Eyes,0.729,0.763,0.0345,0.6680,0.026400,0.1470,0.1130,137029,125.989,42
3,Can't Get You Out of My Head,0.642,0.610,0.0992,0.1600,0.000007,0.4080,0.2940,168000,129.886,52
4,Pieces,0.698,0.780,0.0626,0.1570,0.032500,0.3090,0.2030,177956,122.076,42
5,Paint It Black,0.690,0.741,0.0363,0.0433,0.004290,0.0975,0.0774,146190,125.985,56
6,Free Falling Love,0.771,0.932,0.0424,0.3110,0.000158,0.1160,0.1390,155000,120.012,58
7,Say It Right,0.764,0.793,0.0709,0.0132,0.000980,0.3190,0.1270,129677,124.084,50
8,Creep,0.615,0.829,0.0421,0.0555,0.001560,0.4260,0.0754,203226,124.013,65
9,Apologize,0.677,0.710,0.0552,0.5320,0.000000,0.3370,0.3250,129410,126.109,48


group 19
[0.649 0.515 0.619 0.119 0.004 0.169 0.491 0.453 0.789 0.741]


,track_name,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,tempo,track_pop
0,TO THE MOON,0.745,0.650,0.346,0.0451,0.007580,0.1110,0.386,152137,144.047,79
1,Long Road,0.738,0.685,0.409,0.1350,0.000000,0.4000,0.663,204000,162.039,52
2,Light Switch,0.690,0.626,0.306,0.1050,0.000335,0.0881,0.912,185681,184.009,79
3,I AM WOMAN,0.646,0.471,0.157,0.1220,0.000000,0.1250,0.455,232813,170.084,76
4,MONEY,0.831,0.554,0.218,0.1610,0.000061,0.1520,0.396,168228,140.026,79
5,So Pretty,0.799,0.543,0.240,0.0512,0.042500,0.1340,0.250,85742,187.906,66
6,When Hello Means Goodbye,0.741,0.475,0.205,0.0214,0.000000,0.3510,0.597,158160,166.016,44
7,Hawái,0.783,0.727,0.389,0.1850,0.000025,0.0626,0.558,199112,180.067,77
8,"Gravity (feat. Tyler, The Creator)",0.539,0.615,0.252,0.1730,0.005600,0.1440,0.493,214693,163.924,72
9,WAP (feat. Megan Thee Stallion),0.935,0.454,0.375,0.0194,0.000000,0.0824,0.357,187541,133.073,80


In [ ]:
# В 6 задании приведен пример программы с использованием машинного обучения, где программа делит строки на 20(=к) 
# кластеров через предсказание. Метод к-средних позволяет создать группы точек данных с похожимы характеристиками в 
# датасете.Помимо метода К средних, существует метод К-ближайших соседей.
# 
